In [1]:
import numpy as np
import pandas as pd

In [23]:
base = pd.read_csv(filepath_or_buffer='/Users/Nathan/Desktop/Machine Learning/Projeto de Extensão/FEATURES/FiltredBDS.txt', sep = '\t')


In [24]:
mediana = np.median(base['Best_T'].values)
mediana


22.0

In [25]:
#categorizando o gênero
from sklearn.preprocessing import LabelEncoder
laberencoder= LabelEncoder()
base['Gender'] = laberencoder.fit_transform(base['Gender'])

In [26]:
#FEATURES DO RESNET
resnetfeatures = pd.read_csv(filepath_or_buffer='/Users/Nathan/Desktop/Machine Learning/Projeto de Extensão/2020/Outubro/heat_map_encoded.txt', sep = '\t')
resnetfeatures = resnetfeatures.reindex()
resnetfeatures.drop('Unnamed: 0',inplace = True, axis = 1)

#UNINDO AS FEATURES
base = base.join(resnetfeatures)

base = base.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Subject', 'AgeGroup',
       'FootLen', 'Nationality','BMI', 'SkinColor',
       'Ystudy', 'Footwear', 'Illness', 'Illness2', 'Nmedication',
       'Medication', 'Ortho-Prosthesis', 'Ortho-Prosthesis2', 'Disability',
       'Disability2', 'Falls12m', 'FES_1', 'FES_2', 'FES_3', 'FES_4', 'FES_5',
       'FES_6', 'FES_7', 'FES_T', 'FES_S', 'IPAQ_1a', 'IPAQ_1b', 'IPAQ_2a',
       'IPAQ_2b', 'IPAQ_3a', 'IPAQ_3b', 'IPAQ_4a', 'IPAQ_4b', 'IPAQ_S',
       'TMT_timeA', 'TMT_errorsA', 'TMT_timeB', 'TMT_errorsB', 'Best_1',
       'Best_2', 'Best_3l', 'Best_3r', 'Best_4', 'Best_5', 'Best_6l',
       'Best_6r', 'Best_7', 'Best_8', 'Best_9', 'Best_10', 'Best_11',
       'Best_12', 'Best_13', 'Best_14','x_entropy_cop - Open - Firm','y_entropy_cop - Open - Firm',
       'x_entropy_cop - Closed - Firm','y_entropy_cop - Closed - Firm', 'x_entropy_cop - Open - Foam',
        'y_entropy_cop - Open - Foam','x_entropy_cop - Closed - Foam','y_entropy_cop - Closed - Foam',
        'x_impulse_factor_cop - Open - Firm','y_impulse_factor_cop - Open - Firm',
        'x_impulse_factor_cop - Open - Foam','y_impulse_factor_cop - Open - Foam',
        'x_impulse_factor_cop - Closed - Firm','y_impulse_factor_cop - Closed - Firm',
        'x_kurtosis_cop - Open - Firm','y_kurtosis_cop - Open - Firm',
        'x_kurtosis_cop - Open - Foam','y_kurtosis_cop - Open - Foam',
        'x_kurtosis_cop - Closed - Firm','y_kurtosis_cop - Closed - Firm',
        'x_kurtosis_factor_cop - Open - Firm','y_kurtosis_factor_cop - Open - Firm',
        'x_kurtosis_factor_cop - Open - Foam','y_kurtosis_factor_cop - Open - Foam',
        'x_kurtosis_factor_cop - Closed - Firm','y_kurtosis_factor_cop - Closed - Firm'
        ], axis = 1)



In [27]:
base.iloc[:,4] 

0      25
1      25
2      27
3      26
4      27
       ..
153    28
154    24
155    25
156    26
157    26
Name: Best_T, Length: 158, dtype: int64

In [28]:
#Criando a base de dados abaixo da mediana
baseA = base.copy()
for n in range(len(base.index)):
    if base.iloc[n,4] > mediana:
        baseA.drop(labels=n, inplace = True)
        
#É necessário resetar o indice, pois o KFOLD precisa dos indices do row sequencialmente dispostos        
baseA.reset_index(inplace= True)
baseA.drop('index', inplace= True, axis=1)

#atribui as entradas
xA = baseA.drop(['Best_T'],axis=1)

#normaliza as entradas
#NORMALIZANDO O DATAFRAME
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xA = scaler.fit_transform(xA)

xA = pd.DataFrame(xA)

yA = baseA['Best_T']

In [32]:
yA.shape[0] - xA.shape[0]

0

In [33]:
#Criando a base de dados abaixo da mediana
baseB = base.copy()
for n in range(len(base.index)):
    if base.iloc[n,4] <= mediana:
        baseB.drop(labels=n, inplace = True)
        
#É necessário resetar o indice, pois o KFOLD precisa dos indices do row sequencialmente dispostos        
baseB.reset_index(inplace= True)
baseB.drop('index', inplace= True, axis=1)

#atribui as entradas
xB = baseB.drop(['Best_T'],axis=1)

#NORMALIZANDO O DATAFRAME
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xB = scaler.fit_transform(xB)

xB = pd.DataFrame(xB)

yB = baseB['Best_T']

In [34]:
yB.shape[0] - xB.shape[0]

0

# SVM BASE A

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

parametros = {'kernel':('poly', 'rbf', 'sigmoid'), 'epsilon':[(x/10) for x in range(1,10,1)]}

svrq = SVR()
clf = GridSearchCV(svrq, parametros, scoring='neg_mean_absolute_error', cv= 10)

clf.fit(xA,yA)




GridSearchCV(cv=10, estimator=SVR(),
             param_grid={'epsilon': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,
                                     0.9],
                         'kernel': ('poly', 'rbf', 'sigmoid')},
             scoring='neg_mean_absolute_error')

In [36]:
print('Melhor estimator',clf.best_estimator_)

Melhor estimator SVR(epsilon=0.7, kernel='sigmoid')


In [37]:
bestmaeA=-clf.best_score_
print('Melhor MAE',bestmaeA)

Melhor MAE 2.020474469595178


# SVM BASE B

In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

parametros = {'kernel':('poly', 'rbf', 'sigmoid'), 'epsilon':[(x/10) for x in range(1,10,1)]}

svrq = SVR()
clf = GridSearchCV(svrq, parametros, scoring='neg_mean_absolute_error', cv= 10)

clf.fit(xB,yB)


GridSearchCV(cv=10, estimator=SVR(),
             param_grid={'epsilon': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,
                                     0.9],
                         'kernel': ('poly', 'rbf', 'sigmoid')},
             scoring='neg_mean_absolute_error')

In [39]:
print('Melhor estimator',clf.best_estimator_)

Melhor estimator SVR(epsilon=0.9, kernel='poly')


In [40]:
bestmaeB=-clf.best_score_
print('Melhor MAE',bestmaeB)

Melhor MAE 1.1826277709652948
